# Data Cleaning

This is the notebook where we cleaned our data. It wasn't a straightforward procedure, it required iteration between data cleaning and feature engineering.

In [1]:
# Import libraries needed
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
from scipy import mean

In [2]:
df = pd.read_csv("data/IPOPHIL_patent_data_set.csv")
df = pd.DataFrame(df)
df.head()

,Title,Original Filing Number,Filing Date,Registration Number,Registration Date,Publication Date,IPC Classes,Applicant,Inventor Name(s),Application Subtype,Status
0,PICTURE REGENERATION STABILIZING CIRCUITS,A/1/0/042031,1991.02.20,1042031.0,1991.02.20,NaN,NaN,JEN-MUM-PARK,NaN,Invention,Published
1,PRESS-ON CLOSURE WITH PEELABLE END PANEL,A/1/1993/046127,1993.04.29,1046127.0,2002.12.17,NaN,B65D 41/00\nB65D 43/00\nB65D 43/02,"SAUER, DONALD G.\nFRASER, ROBERT W.\nROTH, DON...",NaN,Invention,Expired
2,"IMIDAZOLE, TRIAZOLE AND TETRAZOLE DERIVATIVES",A/1/1993/046622,1993.07.30,1046622.0,2004.03.15,NaN,A61K 31/41\nA61K 31/42\nA61K 31/44\nA61P 25/00...,"MATASSA, VICTOR G.\nBAKER, RAYMOND\nSTREET, LE...",NaN,Invention,Registered
3,PREFILLED SYRINGE,A/1/1993/047573,1993.04.19,1047573.0,2003.03.11,NaN,A61M 37/00,"TAKEDA CHEMICAL INDUSTRIES, LTD.","TANAKA, NOBUO\nOHTANI, SUIJI\nKISHIMOTO, JOTAR...",Invention,Expired
4,RECORDING MEDIUM MANAGEMENT METHOD WHERE RECOR...,A/1/1994/049461,1994.11.28,1049461.0,2003.09.10,NaN,G11B 20/12\nG11B 27/11\nG11B 27/32\nG06F 17/30...,KATSUYUKI TERANISHI\nTATSUYA IGARASHI,NaN,Invention,Registered


### Data Preprocessing

Here we do regular data cleaning activities - rename faulty values, remove rows which is not essential to our study, and remove rows with null values for important features

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99700 entries, 0 to 99699
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Title                   99378 non-null  object 
 1   Original Filing Number  99700 non-null  object 
 2   Filing Date             99378 non-null  object 
 3   Registration Number     72417 non-null  float64
 4   Registration Date       72368 non-null  object 
 5   Publication Date        59406 non-null  object 
 6   IPC Classes             95659 non-null  object 
 7   Applicant               99697 non-null  object 
 8   Inventor Name(s)        55172 non-null  object 
 9   Application Subtype     99378 non-null  object 
 10  Status                  99700 non-null  object 
dtypes: float64(1), object(10)
memory usage: 8.4+ MB


In [4]:
# drop rows where Title is not available. These are newly filed patents.
df = df[df['Title'].notna()]

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99378 entries, 0 to 99699
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Title                   99378 non-null  object 
 1   Original Filing Number  99378 non-null  object 
 2   Filing Date             99378 non-null  object 
 3   Registration Number     72417 non-null  float64
 4   Registration Date       72368 non-null  object 
 5   Publication Date        59406 non-null  object 
 6   IPC Classes             95659 non-null  object 
 7   Applicant               99375 non-null  object 
 8   Inventor Name(s)        55172 non-null  object 
 9   Application Subtype     99378 non-null  object 
 10  Status                  99378 non-null  object 
dtypes: float64(1), object(10)
memory usage: 9.1+ MB


In [6]:
df['Application Subtype'].value_counts()

Invention            48292
Invention PCT        33462
Utility model        17576
Utility model PCT       48
Name: Application Subtype, dtype: int64

In [7]:
df['Status'].value_counts()

Expired       35905
Published     25029
Inactive      22687
Registered    15351
Filed           285
Withdrawn        45
Converted        19
634              18
630              13
635              10
471              10
636               2
633               1
4                 1
640               1
643               1
Name: Status, dtype: int64

In [8]:
# replace all numbered cells to To Be Cleaned
values_status = [634, 630, 471, 635, 636, 4, 643, 633, 640]

for i in range(len(values_status)):
    df["Status"].replace({str(values_status[i]): "To be cleaned"}, inplace=True)

df['Status'].value_counts()

Expired          35905
Published        25029
Inactive         22687
Registered       15351
Filed              285
To be cleaned       57
Withdrawn           45
Converted           19
Name: Status, dtype: int64

In [9]:
# if there is a date under Registration date, then it is Registered. If none, then it is published. I double checked the data and they fall into either of the two.

df.loc[df['Status'].eq('To be cleaned') & df['Registration Date'].isna(), 'Status'] = 'Published'
df.loc[df['Status'].eq('To be cleaned') & df['Registration Date'].notna(), 'Status'] = 'Registered'

df['Status'].value_counts()

Expired       35905
Published     25071
Inactive      22687
Registered    15366
Filed           285
Withdrawn        45
Converted        19
Name: Status, dtype: int64

In [10]:
# This code removes the data points with a status of Withdrawn and Converted
df = df[df['Status'] != 'Withdrawn']
df = df[df['Status'] != 'Converted']
df['Status'].value_counts()

Expired       35905
Published     25071
Inactive      22687
Registered    15366
Filed           285
Name: Status, dtype: int64

In [11]:
# This code removes the data points with Invention PCT and Utility Model PCT (focus on local filings)
df = df[df['Application Subtype'] != 'Invention PCT']
df = df[df['Application Subtype'] != 'Utility model PCT']
df['Application Subtype'].value_counts()

Invention        48253
Utility model    17576
Name: Application Subtype, dtype: int64

In [12]:
# Removing the registration number 
df.drop(['Registration Number'], axis=1)

,Title,Original Filing Number,Filing Date,Registration Date,Publication Date,IPC Classes,Applicant,Inventor Name(s),Application Subtype,Status
0,PICTURE REGENERATION STABILIZING CIRCUITS,A/1/0/042031,1991.02.20,1991.02.20,NaN,NaN,JEN-MUM-PARK,NaN,Invention,Published
1,PRESS-ON CLOSURE WITH PEELABLE END PANEL,A/1/1993/046127,1993.04.29,2002.12.17,NaN,B65D 41/00\nB65D 43/00\nB65D 43/02,"SAUER, DONALD G.\nFRASER, ROBERT W.\nROTH, DON...",NaN,Invention,Expired
2,"IMIDAZOLE, TRIAZOLE AND TETRAZOLE DERIVATIVES",A/1/1993/046622,1993.07.30,2004.03.15,NaN,A61K 31/41\nA61K 31/42\nA61K 31/44\nA61P 25/00...,"MATASSA, VICTOR G.\nBAKER, RAYMOND\nSTREET, LE...",NaN,Invention,Registered
3,PREFILLED SYRINGE,A/1/1993/047573,1993.04.19,2003.03.11,NaN,A61M 37/00,"TAKEDA CHEMICAL INDUSTRIES, LTD.","TANAKA, NOBUO\nOHTANI, SUIJI\nKISHIMOTO, JOTAR...",Invention,Expired
4,RECORDING MEDIUM MANAGEMENT METHOD WHERE RECOR...,A/1/1994/049461,1994.11.28,2003.09.10,NaN,G11B 20/12\nG11B 27/11\nG11B 27/32\nG06F 17/30...,KATSUYUKI TERANISHI\nTATSUYA IGARASHI,NaN,Invention,Registered
...,...,...,...,...,...,...,...,...,...,...
97735,INCIDENT COMMAND AND CONTROL SYSTEM FOR INFORM...,PH/1/2017/000011,2017.01.10,NaN,2018.07.16,G01V 1/22\nG01V 1/40\nG06Q 10/00\nG06Q 10/10\n...,"CONROY, Teresita Villena-Mendoza","CONROY, Teresita Villena-Mendoza\nUY, Jessica ...",Invention,Published
97736,INTAKE MANIFOLD AND ENGINE INCLUDING INTAKE MA...,PH/1/2017/000012,2017.01.11,NaN,2018.07.16,F02M 35/104\nF02M 35/00,TOYOTA JIDOSHA KABUSHIKI KAISHA,YUKI AMAMOTO\nTakahiro GOTO\nNorihiko SUMI,Invention,Published
97737,MULTIFUNCTION PERIPHERAL,PH/1/2017/000013,2017.01.11,NaN,2018.10.08,H04N 1/00\nB41J 29/00,SEIKO EPSON CORPORATION,"NISHI, Nobuyuki\nOKAWA, Soshi\nKUDO, Shoma\nKA...",Invention,Published
97738,ROBOTIC APPARATUS FOR CONVEYING AND RETREIVING...,PH/1/2017/000014,2017.01.13,NaN,2018.07.23,B65G 15/00\nB25J 9/00,"MONTEJO, ENGR. MARIO G.","MONTEJO, ENGR. MARIO G.",Invention,Published


In [13]:
# Rearranging data columns
df = df[['Original Filing Number','Title','Application Subtype', 'Applicant', 'Inventor Name(s)', 'IPC Classes', 'Filing Date', 'Publication Date', 'Registration Date', 'Status']]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 65829 entries, 0 to 97739
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Original Filing Number  65829 non-null  object
 1   Title                   65829 non-null  object
 2   Application Subtype     65829 non-null  object
 3   Applicant               65826 non-null  object
 4   Inventor Name(s)        23559 non-null  object
 5   IPC Classes             62324 non-null  object
 6   Filing Date             65829 non-null  object
 7   Publication Date        35306 non-null  object
 8   Registration Date       54233 non-null  object
 9   Status                  65829 non-null  object
dtypes: object(10)
memory usage: 5.5+ MB


In [14]:
# drop inventor names bec of a lot of missing information
df = df.drop(['Inventor Name(s)'], axis=1)

In [15]:
# drop rows with missing IPC classes and applicant names
df = df[df['IPC Classes'].notna()]
df = df[df['Applicant'].notna()]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62321 entries, 1 to 97739
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Original Filing Number  62321 non-null  object
 1   Title                   62321 non-null  object
 2   Application Subtype     62321 non-null  object
 3   Applicant               62321 non-null  object
 4   IPC Classes             62321 non-null  object
 5   Filing Date             62321 non-null  object
 6   Publication Date        33649 non-null  object
 7   Registration Date       51931 non-null  object
 8   Status                  62321 non-null  object
dtypes: object(9)
memory usage: 4.8+ MB


In [16]:
df = df.reset_index(drop=True)

In [17]:
df

,Original Filing Number,Title,Application Subtype,Applicant,IPC Classes,Filing Date,Publication Date,Registration Date,Status
0,A/1/1993/046127,PRESS-ON CLOSURE WITH PEELABLE END PANEL,Invention,"SAUER, DONALD G.\nFRASER, ROBERT W.\nROTH, DON...",B65D 41/00\nB65D 43/00\nB65D 43/02,1993.04.29,NaN,2002.12.17,Expired
1,A/1/1993/046622,"IMIDAZOLE, TRIAZOLE AND TETRAZOLE DERIVATIVES",Invention,"MATASSA, VICTOR G.\nBAKER, RAYMOND\nSTREET, LE...",A61K 31/41\nA61K 31/42\nA61K 31/44\nA61P 25/00...,1993.07.30,NaN,2004.03.15,Registered
2,A/1/1993/047573,PREFILLED SYRINGE,Invention,"TAKEDA CHEMICAL INDUSTRIES, LTD.",A61M 37/00,1993.04.19,NaN,2003.03.11,Expired
3,A/1/1994/049461,RECORDING MEDIUM MANAGEMENT METHOD WHERE RECOR...,Invention,KATSUYUKI TERANISHI\nTATSUYA IGARASHI,G11B 20/12\nG11B 27/11\nG11B 27/32\nG06F 17/30...,1994.11.28,NaN,2003.09.10,Registered
4,A/1/1994/049647,HERBICIDAL COMPOSITIONS COMPRISING GLYPHOSATE ...,Invention,"KASSEBAUM, JAMES WEB\nBERK, HOWARD CARY",A01N 25/02\nA01N 25/30\nA01N 33/00\nA01N 33/12...,1994.12.22,NaN,1999.11.12,Inactive
...,...,...,...,...,...,...,...,...,...
62316,PH/1/2017/000011,INCIDENT COMMAND AND CONTROL SYSTEM FOR INFORM...,Invention,"CONROY, Teresita Villena-Mendoza",G01V 1/22\nG01V 1/40\nG06Q 10/00\nG06Q 10/10\n...,2017.01.10,2018.07.16,NaN,Published
62317,PH/1/2017/000012,INTAKE MANIFOLD AND ENGINE INCLUDING INTAKE MA...,Invention,TOYOTA JIDOSHA KABUSHIKI KAISHA,F02M 35/104\nF02M 35/00,2017.01.11,2018.07.16,NaN,Published
62318,PH/1/2017/000013,MULTIFUNCTION PERIPHERAL,Invention,SEIKO EPSON CORPORATION,H04N 1/00\nB41J 29/00,2017.01.11,2018.10.08,NaN,Published
62319,PH/1/2017/000014,ROBOTIC APPARATUS FOR CONVEYING AND RETREIVING...,Invention,"MONTEJO, ENGR. MARIO G.",B65G 15/00\nB25J 9/00,2017.01.13,2018.07.23,NaN,Published


## Feature Engineering - IPC Classes

Here we add more columns that could be derived from IPC Class information

In [18]:
# IPC Classes - this code splits the classes per application
df_ipc = df.set_index(['Original Filing Number','Title', 'Application Subtype','Applicant','Filing Date', 'Publication Date','Registration Date', 'Status']).apply(lambda x: x.str.split('\n').explode()).reset_index() 
df_ipc

,Original Filing Number,Title,Application Subtype,Applicant,Filing Date,Publication Date,Registration Date,Status,IPC Classes
0,A/1/1993/046127,PRESS-ON CLOSURE WITH PEELABLE END PANEL,Invention,"SAUER, DONALD G.\nFRASER, ROBERT W.\nROTH, DON...",1993.04.29,NaN,2002.12.17,Expired,B65D 41/00
1,A/1/1993/046127,PRESS-ON CLOSURE WITH PEELABLE END PANEL,Invention,"SAUER, DONALD G.\nFRASER, ROBERT W.\nROTH, DON...",1993.04.29,NaN,2002.12.17,Expired,B65D 43/00
2,A/1/1993/046127,PRESS-ON CLOSURE WITH PEELABLE END PANEL,Invention,"SAUER, DONALD G.\nFRASER, ROBERT W.\nROTH, DON...",1993.04.29,NaN,2002.12.17,Expired,B65D 43/02
3,A/1/1993/046622,"IMIDAZOLE, TRIAZOLE AND TETRAZOLE DERIVATIVES",Invention,"MATASSA, VICTOR G.\nBAKER, RAYMOND\nSTREET, LE...",1993.07.30,NaN,2004.03.15,Registered,A61K 31/41
4,A/1/1993/046622,"IMIDAZOLE, TRIAZOLE AND TETRAZOLE DERIVATIVES",Invention,"MATASSA, VICTOR G.\nBAKER, RAYMOND\nSTREET, LE...",1993.07.30,NaN,2004.03.15,Registered,A61K 31/42
...,...,...,...,...,...,...,...,...,...
356325,PH/1/2017/000013,MULTIFUNCTION PERIPHERAL,Invention,SEIKO EPSON CORPORATION,2017.01.11,2018.10.08,NaN,Published,B41J 29/00
356326,PH/1/2017/000014,ROBOTIC APPARATUS FOR CONVEYING AND RETREIVING...,Invention,"MONTEJO, ENGR. MARIO G.",2017.01.13,2018.07.23,NaN,Published,B65G 15/00
356327,PH/1/2017/000014,ROBOTIC APPARATUS FOR CONVEYING AND RETREIVING...,Invention,"MONTEJO, ENGR. MARIO G.",2017.01.13,2018.07.23,NaN,Published,B25J 9/00
356328,PH/1/2017/000015,"COPPER FOIL, COPPER-CLAD LAMINATE BOARD, METHO...",Invention,JX Nippon Mining & Metals Corporation,2017.01.13,2018.08.06,NaN,Published,H05K 3/00


In [19]:
import re

df_ipc['IPC Classes'] = df_ipc['IPC Classes'].str.upper()

main = []
sub_main_1 = []
sub_main_2 = []

for values in df_ipc['IPC Classes']:
    main.append(re.search(r'^\D', values).group())
    sub_main_1.append(re.search(r'^\D\d+', values).group())
    sub_main_2.append(re.search(r'^\D\d+\D', values).group())

df_ipc['Main'] = main
df_ipc['Sub Main 1'] = sub_main_1
df_ipc['Sub Main 2'] = sub_main_2

df_ipc

,Original Filing Number,Title,Application Subtype,Applicant,Filing Date,Publication Date,Registration Date,Status,IPC Classes,Main,Sub Main 1,Sub Main 2
0,A/1/1993/046127,PRESS-ON CLOSURE WITH PEELABLE END PANEL,Invention,"SAUER, DONALD G.\nFRASER, ROBERT W.\nROTH, DON...",1993.04.29,NaN,2002.12.17,Expired,B65D 41/00,B,B65,B65D
1,A/1/1993/046127,PRESS-ON CLOSURE WITH PEELABLE END PANEL,Invention,"SAUER, DONALD G.\nFRASER, ROBERT W.\nROTH, DON...",1993.04.29,NaN,2002.12.17,Expired,B65D 43/00,B,B65,B65D
2,A/1/1993/046127,PRESS-ON CLOSURE WITH PEELABLE END PANEL,Invention,"SAUER, DONALD G.\nFRASER, ROBERT W.\nROTH, DON...",1993.04.29,NaN,2002.12.17,Expired,B65D 43/02,B,B65,B65D
3,A/1/1993/046622,"IMIDAZOLE, TRIAZOLE AND TETRAZOLE DERIVATIVES",Invention,"MATASSA, VICTOR G.\nBAKER, RAYMOND\nSTREET, LE...",1993.07.30,NaN,2004.03.15,Registered,A61K 31/41,A,A61,A61K
4,A/1/1993/046622,"IMIDAZOLE, TRIAZOLE AND TETRAZOLE DERIVATIVES",Invention,"MATASSA, VICTOR G.\nBAKER, RAYMOND\nSTREET, LE...",1993.07.30,NaN,2004.03.15,Registered,A61K 31/42,A,A61,A61K
...,...,...,...,...,...,...,...,...,...,...,...,...
356325,PH/1/2017/000013,MULTIFUNCTION PERIPHERAL,Invention,SEIKO EPSON CORPORATION,2017.01.11,2018.10.08,NaN,Published,B41J 29/00,B,B41,B41J
356326,PH/1/2017/000014,ROBOTIC APPARATUS FOR CONVEYING AND RETREIVING...,Invention,"MONTEJO, ENGR. MARIO G.",2017.01.13,2018.07.23,NaN,Published,B65G 15/00,B,B65,B65G
356327,PH/1/2017/000014,ROBOTIC APPARATUS FOR CONVEYING AND RETREIVING...,Invention,"MONTEJO, ENGR. MARIO G.",2017.01.13,2018.07.23,NaN,Published,B25J 9/00,B,B25,B25J
356328,PH/1/2017/000015,"COPPER FOIL, COPPER-CLAD LAMINATE BOARD, METHO...",Invention,JX Nippon Mining & Metals Corporation,2017.01.13,2018.08.06,NaN,Published,H05K 3/00,H,H05,H05K


In [20]:
df_ipc[df_ipc['Sub Main 1'] == 'G5']

,Original Filing Number,Title,Application Subtype,Applicant,Filing Date,Publication Date,Registration Date,Status,IPC Classes,Main,Sub Main 1,Sub Main 2
265287,PH/1/2011/000251,METHODS AND APPARATUS TO DISPLAY LOCALIZED PRO...,Invention,"FISHER-ROSEMOUNTSYSTEMS, INC.",2011.07.27,2013.02.11,NaN,Published,G5B 19/04,G,G5,G5B
265290,PH/1/2011/000253,"METHODS, APPARATUS AND ARTICLES OF MANUFACTURE...",Invention,"FISHER-ROSEMOUNT SYSTEMS, INC.",2011.07.29,2013.02.11,NaN,Published,G5B 19/04,G,G5,G5B


In [21]:
df_ipc[df_ipc['Sub Main 1'] == 'G6']

,Original Filing Number,Title,Application Subtype,Applicant,Filing Date,Publication Date,Registration Date,Status,IPC Classes,Main,Sub Main 1,Sub Main 2
265312,PH/1/2011/000225,"IMAGE PROCESSING APPARATUS, RADIATION IMAGING ...",Invention,CANON KABUSHIKI KAISHA,2011.07.04,2013.02.11,NaN,Published,G6T 3/40,G,G6,G6T


In [22]:
df_ipc.loc[df_ipc['Sub Main 1'].eq('G5'), 'Sub Main 1'] = 'G05'
df_ipc.loc[df_ipc['Sub Main 1'].eq('G6'), 'Sub Main 1'] = 'G06'
df_ipc.loc[df_ipc['Sub Main 2'].eq('G5B'), 'Sub Main 2'] = 'G05B'
df_ipc.loc[df_ipc['Sub Main 2'].eq('G6T'), 'Sub Main 2'] = 'G06T'

List Main IPC Classes Sub Main 1 Sub Main 2 (one hot encoding)

In [23]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(dtype=int, sparse=True)

nominals = pd.DataFrame(encoder.fit_transform(df_ipc[['Main','Sub Main 1','Sub Main 2']]).toarray(),\
                       columns = list(df_ipc['Main'].unique())+ list(df_ipc['Sub Main 1'].unique()) + list(df_ipc['Sub Main 2'].unique()))

feature_df = pd.concat([df_ipc,nominals], axis=1)
feature_df = feature_df.drop(columns = ['Main','Sub Main 1','Sub Main 2'])
feature_df.head()

,Original Filing Number,Title,Application Subtype,Applicant,Filing Date,Publication Date,Registration Date,Status,IPC Classes,B,...,F17B,G10C,B61H,C14B,G10F,B68B,G04D,G16Z,B26K,Y04S
0,A/1/1993/046127,PRESS-ON CLOSURE WITH PEELABLE END PANEL,Invention,"SAUER, DONALD G.\nFRASER, ROBERT W.\nROTH, DON...",1993.04.29,NaN,2002.12.17,Expired,B65D 41/00,0,...,0,0,0,0,0,0,0,0,0,0
1,A/1/1993/046127,PRESS-ON CLOSURE WITH PEELABLE END PANEL,Invention,"SAUER, DONALD G.\nFRASER, ROBERT W.\nROTH, DON...",1993.04.29,NaN,2002.12.17,Expired,B65D 43/00,0,...,0,0,0,0,0,0,0,0,0,0
2,A/1/1993/046127,PRESS-ON CLOSURE WITH PEELABLE END PANEL,Invention,"SAUER, DONALD G.\nFRASER, ROBERT W.\nROTH, DON...",1993.04.29,NaN,2002.12.17,Expired,B65D 43/02,0,...,0,0,0,0,0,0,0,0,0,0
3,A/1/1993/046622,"IMIDAZOLE, TRIAZOLE AND TETRAZOLE DERIVATIVES",Invention,"MATASSA, VICTOR G.\nBAKER, RAYMOND\nSTREET, LE...",1993.07.30,NaN,2004.03.15,Registered,A61K 31/41,1,...,0,0,0,0,0,0,0,0,0,0
4,A/1/1993/046622,"IMIDAZOLE, TRIAZOLE AND TETRAZOLE DERIVATIVES",Invention,"MATASSA, VICTOR G.\nBAKER, RAYMOND\nSTREET, LE...",1993.07.30,NaN,2004.03.15,Registered,A61K 31/42,1,...,0,0,0,0,0,0,0,0,0,0


In [24]:
feature_df['IPC Classes'] = 1
feature_df

,Original Filing Number,Title,Application Subtype,Applicant,Filing Date,Publication Date,Registration Date,Status,IPC Classes,B,...,F17B,G10C,B61H,C14B,G10F,B68B,G04D,G16Z,B26K,Y04S
0,A/1/1993/046127,PRESS-ON CLOSURE WITH PEELABLE END PANEL,Invention,"SAUER, DONALD G.\nFRASER, ROBERT W.\nROTH, DON...",1993.04.29,NaN,2002.12.17,Expired,1,0,...,0,0,0,0,0,0,0,0,0,0
1,A/1/1993/046127,PRESS-ON CLOSURE WITH PEELABLE END PANEL,Invention,"SAUER, DONALD G.\nFRASER, ROBERT W.\nROTH, DON...",1993.04.29,NaN,2002.12.17,Expired,1,0,...,0,0,0,0,0,0,0,0,0,0
2,A/1/1993/046127,PRESS-ON CLOSURE WITH PEELABLE END PANEL,Invention,"SAUER, DONALD G.\nFRASER, ROBERT W.\nROTH, DON...",1993.04.29,NaN,2002.12.17,Expired,1,0,...,0,0,0,0,0,0,0,0,0,0
3,A/1/1993/046622,"IMIDAZOLE, TRIAZOLE AND TETRAZOLE DERIVATIVES",Invention,"MATASSA, VICTOR G.\nBAKER, RAYMOND\nSTREET, LE...",1993.07.30,NaN,2004.03.15,Registered,1,1,...,0,0,0,0,0,0,0,0,0,0
4,A/1/1993/046622,"IMIDAZOLE, TRIAZOLE AND TETRAZOLE DERIVATIVES",Invention,"MATASSA, VICTOR G.\nBAKER, RAYMOND\nSTREET, LE...",1993.07.30,NaN,2004.03.15,Registered,1,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
356325,PH/1/2017/000013,MULTIFUNCTION PERIPHERAL,Invention,SEIKO EPSON CORPORATION,2017.01.11,2018.10.08,NaN,Published,1,0,...,0,0,0,0,0,0,0,0,0,0
356326,PH/1/2017/000014,ROBOTIC APPARATUS FOR CONVEYING AND RETREIVING...,Invention,"MONTEJO, ENGR. MARIO G.",2017.01.13,2018.07.23,NaN,Published,1,0,...,0,0,0,0,0,0,0,0,0,0
356327,PH/1/2017/000014,ROBOTIC APPARATUS FOR CONVEYING AND RETREIVING...,Invention,"MONTEJO, ENGR. MARIO G.",2017.01.13,2018.07.23,NaN,Published,1,0,...,0,0,0,0,0,0,0,0,0,0
356328,PH/1/2017/000015,"COPPER FOIL, COPPER-CLAD LAMINATE BOARD, METHO...",Invention,JX Nippon Mining & Metals Corporation,2017.01.13,2018.08.06,NaN,Published,1,0,...,0,0,0,0,0,1,0,0,0,0


In [25]:
class_info = feature_df.groupby(['Original Filing Number']).sum()
class_info

,IPC Classes,B,A,C,G,E,H,F,D,Y,...,F17B,G10C,B61H,C14B,G10F,B68B,G04D,G16Z,B26K,Y04S
Original Filing Number,,,,,,,,,,,,,,,,,,,,,
A/1/1993/046127,3,0,3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A/1/1993/046622,30,10,0,20,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A/1/1993/047573,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A/1/1994/049461,5,0,0,0,0,0,0,5,0,0,...,0,0,0,0,0,0,0,0,0,0
A/1/1994/049647,6,6,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PH/2/2020/050006,2,0,0,0,0,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0
PH/2/2020/050007,2,0,0,0,0,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0
PH/2/2020/050008,2,0,0,0,0,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
ipc_class_total = class_info['IPC Classes']

In [27]:
ipc_dropped = class_info.drop(['IPC Classes'],axis=1)

In [28]:
ipc_dropped[ipc_dropped != 0] = 1
ipc_dropped

,B,A,C,G,E,H,F,D,Y,B65,...,F17B,G10C,B61H,C14B,G10F,B68B,G04D,G16Z,B26K,Y04S
Original Filing Number,,,,,,,,,,,,,,,,,,,,,
A/1/1993/046127,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A/1/1993/046622,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A/1/1993/047573,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A/1/1994/049461,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A/1/1994/049647,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PH/2/2020/050006,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
PH/2/2020/050007,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
PH/2/2020/050008,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
class_info = pd.merge(ipc_class_total, ipc_dropped, left_index=True, right_index=True)
class_info.rename(columns={'IPC Classes':'Number of Unique IPC Classes'}, inplace=True)
class_info

,Number of Unique IPC Classes,B,A,C,G,E,H,F,D,Y,...,F17B,G10C,B61H,C14B,G10F,B68B,G04D,G16Z,B26K,Y04S
Original Filing Number,,,,,,,,,,,,,,,,,,,,,
A/1/1993/046127,3,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A/1/1993/046622,30,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A/1/1993/047573,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A/1/1994/049461,5,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
A/1/1994/049647,6,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PH/2/2020/050006,2,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
PH/2/2020/050007,2,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
PH/2/2020/050008,2,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
df = pd.merge(df, class_info, left_on = 'Original Filing Number', right_index=True, how = 'left')
df

,Original Filing Number,Title,Application Subtype,Applicant,IPC Classes,Filing Date,Publication Date,Registration Date,Status,Number of Unique IPC Classes,...,F17B,G10C,B61H,C14B,G10F,B68B,G04D,G16Z,B26K,Y04S
0,A/1/1993/046127,PRESS-ON CLOSURE WITH PEELABLE END PANEL,Invention,"SAUER, DONALD G.\nFRASER, ROBERT W.\nROTH, DON...",B65D 41/00\nB65D 43/00\nB65D 43/02,1993.04.29,NaN,2002.12.17,Expired,3,...,0,0,0,0,0,0,0,0,0,0
1,A/1/1993/046622,"IMIDAZOLE, TRIAZOLE AND TETRAZOLE DERIVATIVES",Invention,"MATASSA, VICTOR G.\nBAKER, RAYMOND\nSTREET, LE...",A61K 31/41\nA61K 31/42\nA61K 31/44\nA61P 25/00...,1993.07.30,NaN,2004.03.15,Registered,30,...,0,0,0,0,0,0,0,0,0,0
2,A/1/1993/047573,PREFILLED SYRINGE,Invention,"TAKEDA CHEMICAL INDUSTRIES, LTD.",A61M 37/00,1993.04.19,NaN,2003.03.11,Expired,1,...,0,0,0,0,0,0,0,0,0,0
3,A/1/1994/049461,RECORDING MEDIUM MANAGEMENT METHOD WHERE RECOR...,Invention,KATSUYUKI TERANISHI\nTATSUYA IGARASHI,G11B 20/12\nG11B 27/11\nG11B 27/32\nG06F 17/30...,1994.11.28,NaN,2003.09.10,Registered,5,...,0,0,0,0,0,0,0,0,0,0
4,A/1/1994/049647,HERBICIDAL COMPOSITIONS COMPRISING GLYPHOSATE ...,Invention,"KASSEBAUM, JAMES WEB\nBERK, HOWARD CARY",A01N 25/02\nA01N 25/30\nA01N 33/00\nA01N 33/12...,1994.12.22,NaN,1999.11.12,Inactive,6,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62316,PH/1/2017/000011,INCIDENT COMMAND AND CONTROL SYSTEM FOR INFORM...,Invention,"CONROY, Teresita Villena-Mendoza",G01V 1/22\nG01V 1/40\nG06Q 10/00\nG06Q 10/10\n...,2017.01.10,2018.07.16,NaN,Published,7,...,0,0,0,0,0,0,0,0,0,0
62317,PH/1/2017/000012,INTAKE MANIFOLD AND ENGINE INCLUDING INTAKE MA...,Invention,TOYOTA JIDOSHA KABUSHIKI KAISHA,F02M 35/104\nF02M 35/00,2017.01.11,2018.07.16,NaN,Published,2,...,0,0,0,0,0,0,0,0,0,0
62318,PH/1/2017/000013,MULTIFUNCTION PERIPHERAL,Invention,SEIKO EPSON CORPORATION,H04N 1/00\nB41J 29/00,2017.01.11,2018.10.08,NaN,Published,2,...,0,0,0,0,0,0,0,0,0,0
62319,PH/1/2017/000014,ROBOTIC APPARATUS FOR CONVEYING AND RETREIVING...,Invention,"MONTEJO, ENGR. MARIO G.",B65G 15/00\nB25J 9/00,2017.01.13,2018.07.23,NaN,Published,2,...,0,0,0,0,0,0,0,0,0,0


## Top Categories and Applicants

In [33]:
categories = ['A','B','C','D','E','F','G','H','Y']
category_descriptions = ['Human Necessities', 'Performing Operations; Transporting', 'Chemistry; Metallurgy', 'Textiles; Paper', 'Fixed Constructions', 'Mechanical Engineering; Lighting; Heating; Weapons; Blasting', 'Physics', 'Electricity', 'General Tagging']
total_cat = []
for category in categories:
    total_cat.append(df[category].sum())
main_freq = pd.DataFrame({'Main Categories': categories, 'Category Description': category_descriptions, 'Number of Filings': total_cat})
main_freq.sort_values(by=['Number of Filings'], ascending = False)

,Main Categories,Category Description,Number of Filings
1,B,Performing Operations; Transporting,29133
2,C,Chemistry; Metallurgy,22659
0,A,Human Necessities,12075
5,F,Mechanical Engineering; Lighting; Heating; Wea...,5587
7,H,Electricity,5472
3,D,Textiles; Paper,4609
4,E,Fixed Constructions,3033
6,G,Physics,1942
8,Y,General Tagging,6


In [37]:
X = df[['Number of Unique IPC Classes','Number of Unique Main Categories','Number of Unique Sub Categories 1','Number of Unique Sub Categories 2']]
X.to_csv(r'C:\Users\rical\Desktop\class_count.csv', index=False, header=True)

In [38]:
y = df[['Application Subtype']]
y.to_csv(r'C:\Users\rical\Desktop\application_subtype.csv', index=False, header=True)

In [46]:
df.to_csv(r'C:\Users\rical\Desktop\feature_engineering.csv', index=False, header=True)